In [4]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from transformers import AutoTokenizer, BertForSequenceClassification
import torch
import re

# Download the GGUF model properly
def download_model():
    return hf_hub_download(
        repo_id="TheBloke/OpenHermes-2.5-Mistral-7B-GGUF",
        filename="openhermes-2.5-mistral-7b.Q4_K_M.gguf",
        revision="main"
    )

def load_offline_llm():
    print("Downloading mental health support model...")
    model_path = download_model()
    
    print("Initializing model...")
    return Llama(
        model_path=model_path,
        n_ctx=4096,
        n_gpu_layers=35,
        verbose=True  # Keep verbose for Jupyter compatibility
    )

# Load depression classification model
model_path = "./depression_bert_model"
depression_tokenizer = AutoTokenizer.from_pretrained(model_path)
depression_model = BertForSequenceClassification.from_pretrained(model_path)
depression_model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# Initialize LLM
try:
    llm_model = load_offline_llm()
except Exception as e:
    print(f"Error loading model: {e}")
    print("Try manual download from:")
    print("https://huggingface.co/TheBloke/OpenHermes-2.5-Mistral-7B-GGUF")
    exit()

openhermes-2.5-mistral-7b.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

c:\Users\devan\anaconda3\envs\main\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\devan\.cache\huggingface\hub\models--TheBloke--OpenHermes-2.5-Mistral-7B-GGUF. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Initializing model...


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [6]:
# Enhanced text cleaning
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)
    return text.strip()

# Depression prediction with improved preprocessing
def predict_depression(text):
    cleaned = clean_text(text)
    inputs = depression_tokenizer(
        cleaned,
        return_tensors='pt',
        truncation=True,
        max_length=256,
        padding='max_length'
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = depression_model(**inputs)
    
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return probs[0][1].item()

# Optimized response generation with ChatML formatting
def generate_response(user_input, history, risk_level):
    # Dynamic system message based on risk level
    system_messages = {
        'high': "Provide empathetic support, validate feelings, offer crisis resources immediately. "
                 "Prioritize safety and professional help.",
        'moderate': "Ask open-ended questions to understand their situation better. "
                    "Suggest practical coping strategies. Show genuine concern.",
        'low': "Offer general emotional support. Validate experiences and maintain supportive presence."
    }
    
    chatml_prompt = "<|im_start|>system\n"
    chatml_prompt += system_messages[risk_level] + "<|im_end|>\n"
    
    # Add conversation history
    for msg in history[-4:]:  # Keep recent context
        role = 'user' if msg['role'] == 'user' else 'assistant'
        chatml_prompt += f"<|im_start|>{role}\n{msg['content']}<|im_end|>\n"
    
    # Add current message
    chatml_prompt += f"<|im_start|>user\n{user_input}<|im_end|>\n"
    chatml_prompt += "<|im_start|>assistant\n"
    
    # Generate response with optimized parameters
    response = llm_model(
        prompt=chatml_prompt,
        max_tokens=512,
        temperature=0.65,
        top_p=0.85,
        repeat_penalty=1.15,
        stop=["<|im_end|>"]
    )
    
    return response['choices'][0]['text'].split('<|im_end|>')[0].strip()

# Main chat interface with safety features
def chat():
    history = []
    print("\n🌸 Mental Health Companion: I'm here to listen. Let's talk about how you're feeling.")
    
    while True:
        try:
            user_input = input("\nYou: ").strip()
            if not user_input:
                continue
                
            if user_input.lower() in {'quit', 'exit', 'bye'}:
                print("\nRemember: You're not alone. Reach out anytime.")
                break

            # Analyze emotional state
            depression_prob = predict_depression(user_input)
            
            # Determine risk level
            if depression_prob > 0.7:
                risk_level = 'high'
                safety_note = "\n(For your safety, I want to remind you that professional help is available. "
                safety_note += "You're not alone in this.)"
            elif depression_prob > 0.4:
                risk_level = 'moderate'
                safety_note = ""
            else:
                risk_level = 'low'
                safety_note = ""

            # Generate and display response
            response = generate_response(user_input, history, risk_level)
            print(f"\n🌱 Companion: {response}{safety_note}")
            
            # Update conversation history
            history.extend([
                {'role': 'user', 'content': user_input},
                {'role': 'assistant', 'content': response}
            ])
            
            # Maintain conversation context (last 3 exchanges)
            history = history[-6:]

        except KeyboardInterrupt:
            print("\nTake care of yourself. Come back whenever you need support.")
            break

if __name__ == "__main__":
    chat()


🌸 Mental Health Companion: I'm here to listen. Let's talk about how you're feeling.

🌱 Companion: I'm so sorry to hear that you're feeling sad right now. It's really important that you know your emotions are valid, and it's okay to feel this way. Would you like to talk about what might be causing these feelings? Remember, there is always hope for better days ahead.
(For your safety, I want to remind you that professional help is available. You're not alone in this.)


Llama.generate: prefix-match hit



🌱 Companion: I understand that sometimes words can feel insufficient in expressing the depth of emotions we experience. If you ever need someone to listen or if you'd like to share more about what's on your mind, please don't hesitate to reach out. Your feelings are valid and important, and I hope you find the support you need during this time.
(For your safety, I want to remind you that professional help is available. You're not alone in this.)


Llama.generate: prefix-match hit



🌱 Companion: I am so sorry for the loss of your beloved dog. Losing a pet can be incredibly painful, as they become a part of our families and bring us so much joy. It's completely normal to feel overwhelmed with grief during this time. Is there anyone you can talk to or lean on for support? Remember that it's okay to take the time you need to process your feelings and mourn the loss of your furry friend.
(For your safety, I want to remind you that professional help is available. You're not alone in this.)

Remember: You're not alone. Reach out anytime.
